## Description
<p> Netflix is an online movie streaming service. Their customers watching movies,and subsequently rate them. Netflix uses these ratings to inform other recommendations. I will to focus on the first 5 Rocky movies, starring Sylvester Stallone as a boxer who fights to overcome the odds.</p>
<p>I will find the best model to predict the rating of Rocky 5 given the ratings of the previous 4 films with neural nets using more than 1000 possible combinations of the predictor variables by mixing linear, quadratic, logarithmic and various interaction effects.</p>
<p>The best predtictive model is selected on the Mean Square Error </p>

## Datasets

<strong>recommendDB:</strong> consumerID, rating, rockyID
<p>This dataset contains some missing ratings. I will use this dataset to conduct some explanatory analysis
and ultimately create a simple linear regression recommendation model, as well as apply the best neural net model on 
# it that I find using the training and validation sets</p>

<strong>rockyDB - trainingData:</strong> consumerID, Rocky.1, Rocky.2, Rocky.3, Rocky.4, Rocky.5
<p>This training set will be used for a more complex linear regression & neural net recommendation model</p>
 
<strong>rockyDB - validationData:</strong> consumerID, Rocky.1, Rocky.2, Rocky.3, Rocky.4, Rocky.5
<p>This validation set will be used for a more complex linear regression & neural net recommendation model</p>

## Part 1: EXPLANATORY ANALYSIS with recommendDB

In [2]:
dfRecommend <- read.csv(file = "/Users/nile/Documents/UofR/Career/2018/Github Projects/Netflix Recommendation_MachineLearning_R/recommendDB.csv")
dfRockyTrain <- read.csv(file = "/Users/nile/Documents/UofR/Career/2018/Github Projects/Netflix Recommendation_MachineLearning_R/rockyDB - trainingData.csv")
dfRockyValidation <- read.csv(file = "/Users/nile/Documents/UofR/Career/2018/Github Projects/Netflix Recommendation_MachineLearning_R/rockyDB - validationData.csv")

#Defining a data frame that has row for each consumer and fill it with NA.
 
ratingsDF <- data.frame(matrix(NA, nrow = nrow(unique(dfRecommend['consumerID'])), ncol = 6))
colnames(ratingsDF) <- 
    c('consumerID', 'Rocky.1', 'Rocky.2', 'Rocky.3', 'Rocky.4', 'Rocky.5')

# Filling the data frame from above using data from dfRecommend

library('reshape')

ratingsDF['consumerID'] <- unique(dfRecommend['consumerID'])
ratingsDF <- ratingsDF[order('consumerID'),]

ratingsDF2 = cast(dfRecommend, consumerID ~ rockyID, value = 'rating')
colnames(ratingsDF2) <- c('Consumer ID', 'Rocky.1', 'Rocky.2', 'Rocky.3', 'Rocky.4', 'Rocky.5')
head(ratingsDF2)

ERROR: Error in parse(text = x, srcfile = src): <text>:4:8: unexpected symbol
3: dfRockyValidation <- read.csv(file = "/Users/nile/Documents/UofR/Career/2018/Github Projects/Netflix Recommendation_MachineLearning_R/rockyDB - validationData.csv")
4: import warnings
          ^


In [22]:
# Comparing the correlations between the ratings of each of the five movies using the cor command 
# on the new data frame to find which movies are most similar and different in terms of ratings

dfWithoutID <- ratingsDF2[,-1]
correlationsDf <- cor(dfWithoutID, use = 'pairwise')
correlationsDf <- as.data.frame(correlationsDf)
colnames(correlationsDf) <- c('Rocky.1', 'Rocky.2', 'Rocky.3', 'Rocky.4', 'Rocky.5')
correlationsDf
print("Insights: The most similar movies are Rocky 2 and Rocky 3 since correlation is the highest with 0.7009378.The least similar movies are Rocky 1 and Rocky 5.")

#Calculating the mean rating of each movie.  Which is the best? Which is the worst? Answer in the pdf.  

means <- colMeans(dfWithoutID, na.rm = TRUE)
meansdf <- as.data.frame(means)
meansdf
print("Insights: Best movie is Rocky 1 with an average rating of 3.771311. Worst movie is Rocky 5 with an average of 3.033660")

#  Finding consumers who rated all movies. 

AllRatedDF <-
  subset(ratingsDF2,!is.na(Rocky.1 & Rocky.2 & Rocky.3 & Rocky.4 & Rocky.5))

print("Dataframe with consumers who rated all movies")
head(AllRatedDF)

Rocky.1,Rocky.2,Rocky.3,Rocky.4,Rocky.5
1.0000000,0.6011554,0.5163816,0.4793296,0.3905773
0.6011554,1.0000000,0.7009378,0.6441428,0.5352310
0.5163816,0.7009378,1.0000000,0.6983829,0.6096324
0.4793296,0.6441428,0.6983829,1.0000000,0.6225275
0.3905773,0.5352310,0.6096324,0.6225275,1.0000000


[1] "Insights: The most similar movies are Rocky 2 and Rocky 3 since correlation is the highest with 0.7009378.The least similar movies are Rocky 1 and Rocky 5."


means
3.771311
3.659191
3.595816
3.392406
3.033660


[1] "Insights: Best movie is Rocky 1 with an average rating of 3.771311. Worst movie is Rocky 5 with an average of 3.033660"
[1] "Dataframe with consumers who rated all movies"


,Consumer ID,Rocky.1,Rocky.2,Rocky.3,Rocky.4,Rocky.5
22,609,5,4,4,5,5
58,1693,4,4,5,5,4
68,2000,5,4,3,4,1
77,2338,5,5,5,5,5
80,2469,5,5,5,5,5
85,2787,5,4,4,4,1


## Part 2: PREDICTIVE ANALYSIS for RECOMMENDATION 

<p>Here I will try to determine whether Rocky 5 should be recommended to Netflix customers, based on their ratings of Rocky 1 through 4.  The dependent variables is the rating of Rocky 5, and the independent variables are the ratings of the previous 4 Rocky movies.</p>

In [23]:
# Simple linear regression using AllRatedDF(Dataframe with consumers who rated all movies)

fit = lm(Rocky.5 ~ Rocky.1 + Rocky.2 + Rocky.3 + Rocky.4, data = AllRatedDF)
summary(fit)


Call:
lm(formula = Rocky.5 ~ Rocky.1 + Rocky.2 + Rocky.3 + Rocky.4, 
    data = AllRatedDF)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.2802 -0.5104  0.1754  0.7706  3.0394 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.12873    0.08525  -1.510 0.131118    
Rocky.1     -0.08890    0.02464  -3.608 0.000312 ***
Rocky.2      0.18687    0.02752   6.791 1.27e-11 ***
Rocky.3      0.35226    0.02589  13.606  < 2e-16 ***
Rocky.4      0.41378    0.02117  19.545  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.9665 on 4373 degrees of freedom
Multiple R-squared:  0.4164,	Adjusted R-squared:  0.4159 
F-statistic:   780 on 4 and 4373 DF,  p-value: < 2.2e-16


In [24]:
# Using transformations of the predictor variables (polynomials, log, interactions) for complex models with training and validation datasets

rocky1Vec = c('', '+Rocky.1','+poly(Rocky.1,2)', '+log(Rocky.1)', '+sqrt(Rocky.1)')
rocky2Vec = c('', '+Rocky.2','+poly(Rocky.1,2)', '+log(Rocky.2)', '+sqrt(Rocky.2)')
rocky3Vec = c('', '+Rocky.3','+poly(Rocky.1,2)', '+log(Rocky.3)', '+sqrt(Rocky.3)')
rocky4Vec = c('', '+Rocky.4','+poly(Rocky.1,2)', '+log(Rocky.4)', '+sqrt(Rocky.4)')
formulaSet = paste('Rocky.5~1',
                   apply(
                     expand.grid(rocky1Vec, rocky2Vec, rocky3Vec, rocky4Vec),
                     1,
                     paste,
                     collapse = ''
                   ))

In [30]:
# Creating Mean Squared Error (MSE) function and empty matrix to store MSE of each model above

MSE = function(x) {
  return(mean(x ^ 2))
}

MSEs = matrix(
  data = NA,
  nrow = length(formulaSet),
  ncol = 2,
  byrow = FALSE)
head(MSEs)

NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA


In [29]:
# Linear regressions
for (i in 1:length(formulaSet)) {
  basicLM = lm(as.formula(formulaSet[i]), data = dfRockyTrain)
  MSEs[i, 1] = MSE(dfRockyValidation$Rocky.5 - predict(basicLM, dfRockyValidation))
  MSEs[i, 2] = formulaSet[i]
}
head(MSEs)

Warning message in predict.lm(basicLM, dfRockyValidation):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(basicLM, dfRockyValidation):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(basicLM, dfRockyValidation):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(basicLM, dfRockyValidation):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(basicLM, dfRockyValidation):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(basicLM, dfRockyValidation):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(basicLM, dfRockyValidation):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(basicLM, dfRockyValidation):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(basicLM, dfRockyValidation):
“prediction from a ra

1.57280076474302,Rocky.5~1
1.37033280567815,Rocky.5~1 +Rocky.1
1.36642061207245,"Rocky.5~1 +poly(Rocky.1,2)"
1.3955326384831,Rocky.5~1 +log(Rocky.1)
1.38050641051882,Rocky.5~1 +sqrt(Rocky.1)
1.15041258416435,Rocky.5~1 +Rocky.2


In [31]:
# Neural Nets
library(nnet)

nnetModels <- data.frame(formulaSet)
for(j in 1:length(formulaSet)) {
  set.seed(90)
  basicNNET = nnet(as.formula(formulaSet[j]),data=dfRockyTrain,size = 3, linout = 1,maxit = 1000 )
  nnetModels$mse[j] = MSE(dfRockyValidation$Rocky.5 - predict(basicNNET,dfRockyValidation))
}



# weights:  13
initial  value 44713.860252 
final  value 5620.072538 
converged
# weights:  10
initial  value 46667.125858 
iter  10 value 5483.931447
iter  20 value 4975.033545
iter  30 value 4949.444286
final  value 4948.928327 
converged
# weights:  13
initial  value 45929.898133 
iter  10 value 5346.663945
iter  20 value 4950.821919
iter  30 value 4947.239734
final  value 4946.887767 
converged
# weights:  10
initial  value 47851.725668 
iter  10 value 4981.454398
iter  20 value 4949.774806
iter  30 value 4946.974473
iter  40 value 4946.659816
iter  40 value 4946.659778
iter  40 value 4946.659778
final  value 4946.659778 
converged
# weights:  10
initial  value 47442.939988 
iter  10 value 4987.384320
iter  20 value 4949.221952
iter  30 value 4946.900698
iter  40 value 4946.767504
iter  50 value 4946.677750
iter  60 value 4946.660321
final  value 4946.659775 
converged
# weights:  10
initial  value 46727.830763 
iter  10 value 4074.261906
iter  20 value 4035.826435
iter  30 value 4

In [32]:
# Model which performs best in terms of out of sampleMSE
nnet_min_mse <- nnetModels[which.min(nnetModels$mse),]

nnet_min_mse

,formulaSet,mse
173,"Rocky.5~1 +poly(Rocky.1,2)+sqrt(Rocky.2)+Rocky.3+Rocky.4",0.8909278


In [39]:
#f) Estimate and store the best model using the full dataset.  

set.seed(90)
basicNNET = nnet(as.formula(formulaSet[407]),data=dfRockyTrain,size = 3, linout = 1,maxit = 1000 )
MSE(AllRatedDF$Rocky.5 - predict(basicNNET,AllRatedDF))

bestmodel <- as.character(formulaSet[407])
print("best model")
bestmodel
print("neural net model details")
basicNNET

# weights:  19
initial  value 46566.874842 
iter  10 value 3814.089585
iter  20 value 3297.005969
iter  30 value 3243.864412
iter  40 value 3206.372555
iter  50 value 3175.964126
iter  60 value 3170.518642
iter  70 value 3162.827783
iter  80 value 3159.744817
iter  90 value 3158.778202
iter 100 value 3158.603787
iter 110 value 3157.965463
iter 120 value 3157.715586
iter 130 value 3157.518109
iter 140 value 3157.468093
iter 150 value 3157.292571
iter 160 value 3157.200398
final  value 3157.107355 
converged


[1] 0.8985777

[1] "best model"


[1] "Rocky.5~1 +Rocky.1+Rocky.2+Rocky.3+log(Rocky.4)"

[1] "neural net model details"


a 4-3-1 network with 19 weights
inputs: Rocky.1 Rocky.2 Rocky.3 log(Rocky.4) 
output(s): Rocky.5 
options were - linear output units 